In [2]:
import pandas as pd
import json

In [3]:
# Load your JSON content into a dictionary
with open('response_dict.json', 'r', encoding='utf-8') as file:
    json_data = json.load(file)

In [4]:
splits = {'validation': 'all/validation-00000-of-00001.parquet'}
dataset_df = pd.read_parquet("hf://datasets/cais/mmlu/" + splits["validation"])

C:\Users\sequi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Extract the relevant data from JSON
instructions = []
choices = []
responses_json = []

for key in json_data:
    for entry in json_data[key]:
        # Extraire les instructions, choix et réponses
        instruction_start = entry.find('### Instruction:') + len('### Instruction:')
        instruction_end = entry.find('### Choices:')
        instruction = entry[instruction_start:instruction_end].strip()
        
        choices_start = entry.find('### Choices:') + len('### Choices:')
        choices_end = entry.find('### Response:')
        choice = entry[choices_start:choices_end].strip()
        
        response_start = entry.find('### Response:') + len('### Response:')
        response = entry[response_start:].strip()

        instructions.append(instruction)
        choices.append(choice)
        responses_json.append(response)

In [6]:
# Combine the data
data = {
    'Instruction': instructions,
    'Choix': choices,
    'Réponse Dataset': dataset_df['answer'],  # This assumes the order matches
    'Réponse Json': responses_json
}

In [8]:
import re
# Create the DataFrame
df = pd.DataFrame(data)
def remove_non_printable(text):
    return re.sub(r'[^\x20-\x7E]', '', text)

# Apply the function to the entire DataFrame
df = df.applymap(lambda x: remove_non_printable(str(x)))
# Save to Excel
df.to_excel('result_inference_mmlu.xlsx', index=False)

C:\Users\sequi\AppData\Local\Temp\ipykernel_29164\2742041011.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: remove_non_printable(str(x)))
